In [1]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from concurrent.futures import ProcessPoolExecutor
from tqdm.auto import tqdm

In [2]:
# Get all the data files recursively
path = "/root/robocup2d_data_processed2"
data_list = []
for root, dirs, files in tqdm(
    os.walk(path), total=len([None for _, _, _ in os.walk(path)])
):
    for file in tqdm(files, leave=False):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join(root, file))
            df = df.where(pd.notnull(df), None)  # NaN を None に置換
            int_cols = df.select_dtypes(include=["int64"]).columns
            for col in int_cols:
                df[col] = df[col].astype("float64")
            data_list.append(df)

  0%|          | 0/91 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/2203 [00:00<?, ?it/s]

  0%|          | 0/3220 [00:00<?, ?it/s]

  0%|          | 0/1565 [00:00<?, ?it/s]

...

  0%|          | 0/7640 [00:00<?, ?it/s]

  0%|          | 0/1450 [00:00<?, ?it/s]

  0%|          | 0/2048 [00:00<?, ?it/s]

  0%|          | 0/3742 [00:00<?, ?it/s]

  0%|          | 0/2447 [00:00<?, ?it/s]

In [3]:
len(data_list)

237599

In [4]:
# pd.set_option('display.max_columns', 1000)

data_list[0].dtypes.unique()

# for i in data_list[0].dtypes:
#     print(i)

array([dtype('float64'), dtype('O')], dtype=object)

In [ ]:
def generator(data):
    def _generator():
        for d in tqdm(data):
            yield d

    return _generator


dataset = Dataset.from_generator(generator(data_list))
dataset

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/237599 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/40 [00:00<?, ?it/s]

Dataset({
    features: ['#', 'cycle', 'stopped', 'playmode', 'l_name', 'l_score', 'l_pen_score', 'r_name', 'r_score', 'r_pen_score', 'b_x', 'b_y', 'b_vx', 'b_vy', 'l1_t', 'l1_x', 'l1_y', 'l1_vx', 'l1_vy', 'l1_body', 'l1_neck', 'l1_vwidth', 'l1_stamina', 'l2_t', 'l2_x', 'l2_y', 'l2_vx', 'l2_vy', 'l2_body', 'l2_neck', 'l2_vwidth', 'l2_stamina', 'l3_t', 'l3_x', 'l3_y', 'l3_vx', 'l3_vy', 'l3_body', 'l3_neck', 'l3_vwidth', 'l3_stamina', 'l4_t', 'l4_x', 'l4_y', 'l4_vx', 'l4_vy', 'l4_body', 'l4_neck', 'l4_vwidth', 'l4_stamina', 'l5_t', 'l5_x', 'l5_y', 'l5_vx', 'l5_vy', 'l5_body', 'l5_neck', 'l5_vwidth', 'l5_stamina', 'l6_t', 'l6_x', 'l6_y', 'l6_vx', 'l6_vy', 'l6_body', 'l6_neck', 'l6_vwidth', 'l6_stamina', 'l7_t', 'l7_x', 'l7_y', 'l7_vx', 'l7_vy', 'l7_body', 'l7_neck', 'l7_vwidth', 'l7_stamina', 'l8_t', 'l8_x', 'l8_y', 'l8_vx', 'l8_vy', 'l8_body', 'l8_neck', 'l8_vwidth', 'l8_stamina', 'l9_t', 'l9_x', 'l9_y', 'l9_vx', 'l9_vy', 'l9_body', 'l9_neck', 'l9_vwidth', 'l9_stamina', 'l10_t', 'l10_x',

In [6]:
dataset.push_to_hub("ReonOhashi/RobocupTrajectoryPrediction_8team", private=True)

Uploading the dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ReonOhashi/RobocupTrajectoryPrediction_8team/commit/939d484d29b31c7f7389e451c830fa98fd942284', commit_message='Upload dataset', commit_description='', oid='939d484d29b31c7f7389e451c830fa98fd942284', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ReonOhashi/RobocupTrajectoryPrediction_8team', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ReonOhashi/RobocupTrajectoryPrediction_8team'), pr_revision=None, pr_num=None)